# Trabalho de inteligencia artificial

## Autor
- **Nome**: _Vinicius Zapalá dos Santos_
- **RA**: 3019109027

## Entregas
- [ ] Entrega I
    - leitura dos dados, merge e geração de dataset único.
- [ ] Entrega II
    - Análise exploratória do conjunto de dados e conclusões.
- [ ] Entrega III
    - Modelo preditivo com tuning de parâmetros + Entregas I e II.
- [ ] Entrega IV
    - Deploy do modelo (API ou aplicação) + Entregas I, II e III.

# Importando Libs

In [1]:
import pandas as pd

# Lendo Datasets

In [14]:
#creating variables for folder and files from datasets
dataFolder= "/content/data"

#creating a variable for path files, because their files have a similar name
dataFileName = "/reclamacoes-fundamentadas-sindec-"
defaultPath = dataFolder + dataFileName

#setting datasets
df2012 = pd.read_csv(defaultPath+"2012.csv")
df2013 = pd.read_csv(defaultPath+"2013.csv")
df2014 = pd.read_csv(defaultPath+"2014.csv")
df2015 = pd.read_csv(defaultPath+"2015.csv")
df2016 = pd.read_csv(defaultPath+"2016.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Contatenando datasets

In [15]:
#creating a list for datasets will be concatenated
frames = [df2012, df2013, df2014, df2015, df2016]

#concatenating datasets
dfFull = pd.concat(frames)

In [16]:
dfFull.shape

(1206072, 23)

# Exportando dataset concatenado

In [21]:
dfFull.to_csv(dataFolder+"/exported"+dataFileName+"full.csv")